# 3D detector evaluation

## Notes

#### File Paths:

Ensure that `base_dir` is correctly set to the path where your data is located.

The script assumes the following directory structure:

<pre>
base_dir/
├── detector_labels/
├── gt_test/
│   ├── labels/
│   ├── npy/
│   └── html/
</pre>

#### Dependencies:

The script uses `numpy`, `glob`, `os`, and `plotly`.

Ensure you have `plotly` installed (`pip install plotly`) and that `BoundingBox3D` is properly defined in `bbox_utils`.

#### Visualization in Notebook:

When `show_in_notebook` is `True`, the visualization will be displayed inline if you're running in a Jupyter notebook.

If running in a script or console, this may not display, and you might need to adjust the code accordingly.

#### Saving Visualizations:

If `save_visualizations` is `True`, the script will save HTML files for each visualization.

The files are saved in a subdirectory under `html_path`, named after each file's basename with a `_results` suffix.

#### Adjusting Thresholds:

You can adjust `confidence_threshold` and `iou_threshold` according to your evaluation criteria.


# Convert the pickle results file into txt file:
First - convert the results.pkl file in the output directory to txt files using get_results.py

In [2]:
import os
import numpy as np
import glob
from bbox_utils import BoundingBox3D
import plotly.graph_objects as go

In [3]:
# =====================================================
# Data Loading Functions
# =====================================================

def read_gt_file(fname):
    """
    Reads ground truth bounding boxes from a text file.
    Each line in the file corresponds to one bounding box.

    Parameters:
        fname (str): Path to the ground truth file.

    Returns:
        list: A list of ground truth bounding boxes.
    """
    with open(fname, 'r') as file:
        lines = file.readlines()

    data = []

    for line in lines:
        # Split each line into a list of strings
        values = line.strip().split()

        # Convert numerical values to floats and keep the last element as a string
        values = [float(val) if i < len(values) - 1 else val for i, val in enumerate(values)]

        # Append the list of values to the data list
        data.append(values)
        
    return data

def read_detector_file(fname):
    """
    Reads detected bounding boxes from a text file.
    Each line in the file corresponds to one detection.

    Parameters:
        fname (str): Path to the detector output file.

    Returns:
        list: A list of detected bounding boxes.
    """
    with open(fname, 'r') as file:
        lines = file.readlines()

    data = []

    for line in lines:
        # Split each line into a list of strings
        values = line.strip().split()

        # Convert strings to floats
        values = [float(val) for val in values]

        # Append the list of values to the data list
        data.append(values)
        
    return data

In [4]:
# =====================================================
# Evaluation Functions
# =====================================================

def calculate_iou(box1, box2):
    """
    Calculate IoU (Intersection over Union) between two bounding boxes.

    Parameters:
        box1 (list): Coordinates and dimensions of the first box.
        box2 (list): Coordinates and dimensions of the second box.

    Returns:
        float: The IoU value.
    """
    x1, y1, z1, w1, h1 = box1[:5]
    x2, y2, z2, w2, h2 = box2[:5]

    intersect_x = max(0, min(x1 + w1 / 2, x2 + w2 / 2) - max(x1 - w1 / 2, x2 - w2 / 2))
    intersect_y = max(0, min(y1 + h1 / 2, y2 + h2 / 2) - max(y1 - h1 / 2, y2 - h2 / 2))

    intersection = intersect_x * intersect_y
    union = w1 * h1 + w2 * h2 - intersection

    iou = intersection / max(union, 1e-10)  # Avoid division by zero

    return iou

def evaluate_detection(gt_boxes, dnn_boxes, iou_threshold=0.3):
    """
    Evaluates detections against ground truth boxes.

    Parameters:
        gt_boxes (list): Ground truth bounding boxes.
        dnn_boxes (list): Detected bounding boxes.
        iou_threshold (float): IoU threshold to consider a detection as True Positive.

    Returns:
        list: Per-detection results with confidence and TP/FP flag.
        int: Number of False Negatives.
        int: Total number of ground truth boxes.
    """
    per_detections = []
    matched_gt_boxes = set()
    total_gt_boxes = len(gt_boxes)
    
    # Sort detections by confidence in descending order
    dnn_boxes_sorted = sorted(dnn_boxes, key=lambda x: x[7], reverse=True)
    
    for det_idx, det in enumerate(dnn_boxes_sorted):
        det_confidence = det[7]
        det_box = det
        is_tp = False

        # Match detection to ground truth boxes
        for gt_idx, gt_box in enumerate(gt_boxes):
            if gt_idx in matched_gt_boxes:
                continue  # Skip already matched GT boxes
            if calculate_iou(det_box, gt_box) >= iou_threshold:
                is_tp = True
                matched_gt_boxes.add(gt_idx)
                break  # Break after finding the first match

        per_detections.append({'confidence': det_confidence, 'is_tp': is_tp})

    num_fn = total_gt_boxes - len(matched_gt_boxes)
    return per_detections, num_fn, total_gt_boxes

def compute_ap(per_detections, total_gt_boxes):
    """
    Computes Average Precision (AP) for detections.

    Parameters:
        per_detections (list): List of detections with confidence and TP/FP flag.
        total_gt_boxes (int): Total number of ground truth boxes.

    Returns:
        float: Average Precision (AP) score.
    """
    # Sort detections by confidence in descending order
    per_detections = sorted(per_detections, key=lambda x: x['confidence'], reverse=True)
    
    tp_cumsum = np.cumsum([d['is_tp'] for d in per_detections])
    fp_cumsum = np.cumsum([not d['is_tp'] for d in per_detections])
    
    precisions = tp_cumsum / (tp_cumsum + fp_cumsum + 1e-10)
    recalls = tp_cumsum / (total_gt_boxes + 1e-10)
    
    # Append sentinel values at the start and end
    mrec = np.concatenate(([0.], recalls, [1.]))
    mpre = np.concatenate(([0.], precisions, [0.]))

    # Compute the precision envelope
    for i in range(len(mpre) - 2, -1, -1):
        mpre[i] = np.maximum(mpre[i], mpre[i + 1])

    # Compute AP by integrating over recall levels
    idx = np.where(mrec[1:] != mrec[:-1])[0]  # Points where recall changes
    ap = np.sum((mrec[idx + 1] - mrec[idx]) * mpre[idx + 1])

    return ap

def calculate_metrics(tp, fp, fn):
    """
    Calculates precision, recall, and F1 score.

    Parameters:
        tp (int): True Positives.
        fp (int): False Positives.
        fn (int): False Negatives.

    Returns:
        tuple: Precision, Recall, F1 Score.
    """
    EPSILON = 1e-10  # Avoid division by zero

    precision = tp / (tp + fp + EPSILON)
    recall = tp / (tp + fn + EPSILON)
    f1_score = 2 * (precision * recall) / (precision + recall + EPSILON)

    return precision, recall, f1_score

In [5]:
# =====================================================
# Visualization Function
# =====================================================

def saveHTML(pc, gt_boxes, detector_boxes, file_path=None, show_in_notebook=True, min_confidence=0.0):
    """
    Creates a 3D visualization of point clouds and bounding boxes.

    Parameters:
        pc (numpy.ndarray): Point cloud data.
        gt_boxes (list): Ground truth bounding boxes.
        detector_boxes (list): Detected bounding boxes.
        file_path (str): Path to save the HTML file. If None, the HTML is not saved.
        show_in_notebook (bool): Whether to display the visualization in the notebook.
        min_confidence (float): Minimum confidence threshold to display detections.
    """
    pc = pc.T
    data = []

    # Plot the point cloud
    data.append(go.Scatter3d(
        x=pc[0,:], y=pc[1,:], z=pc[2,:],
        mode="markers",
        marker=dict(size=2, color='mediumturquoise', opacity=1)
    ))

    # Function to format confidence score text
    def format_confidence_text(confidence=None):
        if confidence is not None:
            return f"<b>{confidence*100:.2f}%</b>"
        else:
            return ""

    # Define edges for the bounding boxes
    edges = [
        (0,1), (1,2), (2,3), (3,0),  # Bottom edges
        (4,5), (5,6), (6,7), (7,4),  # Top edges
        (0,4), (1,5), (2,6), (3,7)   # Vertical edges
    ]

    # Add Detector Bounding Boxes in Red with confidence scores
    for det in detector_boxes:
        confidence = det[7]
        if confidence >= min_confidence:
            bbox = BoundingBox3D(
                x=det[0], y=det[1], z=det[2],
                length=det[3], width=det[4], height=det[5],
                euler_angles=[0, 0, det[6]]
            )
            corners = bbox.p  # shape (8,3)

            # Prepare the edges of the box
            x_edges = []
            y_edges = []
            z_edges = []

            for edge in edges:
                start_idx, end_idx = edge
                x_edges += [corners[start_idx, 0], corners[end_idx, 0], None]
                y_edges += [corners[start_idx, 1], corners[end_idx, 1], None]
                z_edges += [corners[start_idx, 2], corners[end_idx, 2], None]

            # Add the edges as lines
            data.append(go.Scatter3d(
                x=x_edges, y=y_edges, z=z_edges,
                mode='lines',
                line=dict(color='red', width=6),
                hovertext=format_confidence_text(confidence),
                hoverinfo='text'
            ))

            # Choose one of the top edges (e.g., edge from corner 4 to corner 5)
            corner_start = corners[4]
            corner_end = corners[5]
            text_x = (corner_start[0] + corner_end[0]) / 2
            text_y = (corner_start[1] + corner_end[1]) / 2
            text_z = (corner_start[2] + corner_end[2]) / 2 + 0.5  # Slightly above the edge

            # Add confidence score text on top of the edge
            data.append(go.Scatter3d(
                x=[text_x],
                y=[text_y],
                z=[text_z],
                mode='text',
                text=[format_confidence_text(confidence)],
                textfont=dict(color='red', size=16),  # Increased font size
                showlegend=False
            ))

    # Add Ground Truth Bounding Boxes in Green
    for gt in gt_boxes:
        bbox = BoundingBox3D(
            x=gt[0], y=gt[1], z=gt[2],
            length=gt[3], width=gt[4], height=gt[5],
            euler_angles=[0, 0, gt[6]]
        )
        corners = bbox.p  # shape (8,3)

        # Prepare the edges of the box
        x_edges = []
        y_edges = []
        z_edges = []

        for edge in edges:
            start_idx, end_idx = edge
            x_edges += [corners[start_idx, 0], corners[end_idx, 0], None]
            y_edges += [corners[start_idx, 1], corners[end_idx, 1], None]
            z_edges += [corners[start_idx, 2], corners[end_idx, 2], None]

        # Add the edges as lines
        data.append(go.Scatter3d(
            x=x_edges, y=y_edges, z=z_edges,
            mode='lines',
            line=dict(color='green', width=6),
            hoverinfo='none'
        ))

    # Define the layout with specified aesthetics
    layout = go.Layout(
        scene=dict(
            xaxis=dict(nticks=40, range=[-20, 20], showbackground=True, backgroundcolor='rgb(30, 30, 30)',
                       gridcolor='rgb(127, 127, 127)', zerolinecolor='rgb(127, 127, 127)'),
            yaxis=dict(nticks=40, range=[-20, 20], showbackground=True, backgroundcolor='rgb(30, 30, 30)',
                       gridcolor='rgb(127, 127, 127)', zerolinecolor='rgb(127, 127, 127)'),
            zaxis=dict(nticks=20, range=[-1, 19], showbackground=True, backgroundcolor='rgb(30, 30, 30)',
                       gridcolor='rgb(127, 127, 127)', zerolinecolor='rgb(127, 127, 127)'),
            xaxis_title="x (meters)",
            yaxis_title="y (meters)",
            zaxis_title="z (meters)"
        ),
        margin=dict(r=10, l=10, b=10, t=10),
        paper_bgcolor='rgb(30, 30, 30)',
        font=dict(family="Courier New, monospace", color='rgb(127, 127, 127)'),
        legend=dict(font=dict(family="Courier New, monospace", color='rgb(127, 127, 127)'))
    )

    fig = go.Figure(data=data, layout=layout)

    if show_in_notebook:
        fig.show()

    if file_path:
        fig.write_html(file_path)

In [6]:
# =====================================================
# Main Processing Function
# =====================================================

def process_files(base_dir, evaluate=True, visualize=False, single_file=None, confidence_threshold=0.3, iou_threshold=0.3, show_in_notebook=False, save_visualizations=False):
    """
    Processes detection and ground truth files to compute evaluation metrics and/or generate visualizations.

    Parameters:
        base_dir (str): Base directory containing detector outputs and ground truth data.
        evaluate (bool): Whether to perform evaluation.
        visualize (bool): Whether to generate visualizations.
        single_file (str): Filename to process a single file (e.g., '00001.txt'). If None, processes all files.
        confidence_threshold (float): Confidence threshold for metrics calculation.
        iou_threshold (float): IoU threshold for determining true positives.
        show_in_notebook (bool): Whether to display visualizations in the notebook.
        save_visualizations (bool): Whether to save visualizations to files.

    Returns:
        dict: A dictionary containing evaluation metrics (if evaluation is performed).
    """
    detector_path = os.path.join(base_dir, 'detector_labels')
    gt_path = os.path.join(base_dir, 'gt_test/labels')
    npy_path = os.path.join(base_dir, 'gt_test/npy')
    html_path = os.path.join(base_dir, 'gt_test/html')

    # Prepare file list
    if single_file:
        file_list = [os.path.join(detector_path, single_file)]
    else:
        file_list = glob.glob(os.path.join(detector_path, '*.txt'))

    all_detections = []
    total_gt_boxes = 0

    for file_name in file_list:
        basename = os.path.splitext(os.path.basename(file_name))[0]
        gt_file = os.path.join(gt_path, f"{basename}.txt")
        npy_file = os.path.join(npy_path, f"{basename}.npy")

        detector_boxes = read_detector_file(file_name)
        gt_boxes = read_gt_file(gt_file)
        pc = np.load(npy_file)

        # Evaluate detections
        if evaluate:
            per_detections, num_fn, num_gt_boxes = evaluate_detection(gt_boxes, detector_boxes, iou_threshold)
            all_detections.extend(per_detections)
            total_gt_boxes += num_gt_boxes

        # Generate visualizations
        if visualize:
            results_dir = os.path.join(html_path, f"{basename}_results")
            if save_visualizations:
                os.makedirs(results_dir, exist_ok=True)
                file_path_html = os.path.join(results_dir, f"{basename}.html")
            else:
                file_path_html = None

            saveHTML(
                pc,
                gt_boxes,
                detector_boxes,
                file_path=file_path_html,
                show_in_notebook=show_in_notebook,
                min_confidence=confidence_threshold
            )

    # Compute evaluation metrics
    if evaluate:
        # Compute mAP using all detections (without confidence threshold)
        ap = compute_ap(all_detections, total_gt_boxes)

        # Apply confidence threshold for precision, recall, and F1 score
        detections_above_threshold = [d for d in all_detections if d['confidence'] >= confidence_threshold]
        tp = sum([1 for d in detections_above_threshold if d['is_tp']])
        fp = sum([1 for d in detections_above_threshold if not d['is_tp']])
        fn = total_gt_boxes - tp  # FN is total ground truths minus TPs

        # Calculate precision, recall, and F1 score
        precision, recall, f1_score = calculate_metrics(tp, fp, fn)

        metrics = {
            'Average Precision (AP)': ap,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1_score,
            'True Positives': tp,
            'False Positives': fp,
            'False Negatives': fn,
            'Total Ground Truths': total_gt_boxes
        }

        return metrics

    return None  # If evaluation is not performed

In [ ]:
# =====================================================
# Example Usage
# =====================================================


# Base directory containing the necessary data
base_dir = '/path/to/your/base_dir'

# Example 1: Evaluate metrics without visualization
metrics = process_files(
    base_dir=base_dir,
    evaluate=True,
    visualize=False,
    confidence_threshold=0.3,
    iou_threshold=0.3
)

if metrics:
    print("Evaluation Metrics:")
    for key, value in metrics.items():
        if isinstance(value, float):
            print(f"{key}: {value:.4f}")
        else:
            print(f"{key}: {value}")

# Example 2: Visualize a single file in the notebook
# process_files(
#      base_dir=base_dir,
#      evaluate=False,
#      visualize=True,
#      single_file='00001.txt',
#      show_in_notebook=True,
#      save_visualizations=False
#  )

# Example 3: Save visualizations for all files
# process_files(
#     base_dir=base_dir,
#     evaluate=False,
#     visualize=True,
#     save_visualizations=True
# )